# Getting the mobility score from transitscreen API

In [1]:
# Hat tip to John Whitesell for parts of this code.

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [3]:
# Read in the file from the previous notebook.
df=pd.read_csv('data/dc_addresses.csv', dtype={'ZIPCODE': object})
print(df.shape)

(4933, 9)


## Preparing the data

In [4]:
# Combine latitude and longitude into a single variable.
df['coordinates'] = df.apply(lambda x: (str(x.LATITUDE)+","+str(x.LONGITUDE)), axis=1)

In [5]:
# Read in the API key from another folder (saved locally, outside of the github repo)
key_path = "/Users/austinlasseter/keys/API_key.txt"
key_file = open(key_path, 'r')
api_key = str(key_file.readline())[:16] # There's a hidden '/n' at the end of that string; slice at index 16 to remove.

In [6]:
# Create 4 strings, for the 4 components in the sample call.
string1 = "https://mobilityscore.transitscreen.io/api"
string2 = "/v1/locations.json?coordinates="
string3 = "&key="
string4 = "&geojson=false"

In [7]:
# Define a function that reads in the coordinates, then produces a new string from those 4 strings.
def api_string(coords):
    new_string = string1+string2+coords+string3+api_key+string4
    return new_string

In [8]:
# Apply the function to our combined lat-long coordinates.
df['call'] = df['coordinates'].apply(api_string)

## Making the API calls

In [9]:
# Extract the information from the transitscreen API. Do this in pieces.
for i in range(500):
    df.loc[i, 'json'] = [requests.get(df.loc[i,'call']).json()] 

In [10]:
# Extract the information from the transitscreen API. Do this in pieces.
for i in range(501,1500):
    df.loc[i, 'json'] = [requests.get(df.loc[i,'call']).json()] 

In [16]:
# Extract the information from the transitscreen API. Do this in pieces.
for i in range(1501,3000):
    df.loc[i, 'json'] = [requests.get(df.loc[i,'call']).json()] 

In [17]:
# Extract the information from the transitscreen API. Do this in pieces.
for i in range(3001,4500):
    df.loc[i, 'json'] = [requests.get(df.loc[i,'call']).json()] 

In [18]:
# Extract the information from the transitscreen API. Do this in pieces.
for i in range(4501,len(df)):
    df.loc[i, 'json'] = [requests.get(df.loc[i,'call']).json()] 

## Results

In [21]:
df.tail()

,LATITUDE,LONGITUDE,FULLADDRESS,ZIPCODE,ASSESSMENT_NBHD,CLUSTER_,WARD,Units,Residential,coordinates,call,json,mobilityscore
4928,38.929740,-76.964123,3032 VISTA STREET NE,20018,Woodridge,Cluster 24,Ward 5,3303,0.224947,"38.92974033,-76.96412279",https://mobilityscore.transitscreen.io/api/v1/...,[{'data': {'mobilityScore': {'latitude': 38.92...,NaN
4929,38.925779,-76.966164,3002 FRANKLIN STREET NE,20018,Woodridge,Cluster 22,Ward 5,3303,0.224947,"38.92577927,-76.96616359",https://mobilityscore.transitscreen.io/api/v1/...,[{'data': {'mobilityScore': {'latitude': 38.92...,NaN
4930,38.938105,-76.971944,2401 RANDOLPH STREET NE,20018,Woodridge,Cluster 24,Ward 5,3303,0.224947,"38.93810506,-76.97194418",https://mobilityscore.transitscreen.io/api/v1/...,[{'data': {'mobilityScore': {'latitude': 38.93...,NaN
4931,38.929389,-76.970928,2430 IRVING STREET NE,20018,Woodridge,Cluster 22,Ward 5,3303,0.224947,"38.92938856,-76.97092764",https://mobilityscore.transitscreen.io/api/v1/...,[{'data': {'mobilityScore': {'latitude': 38.92...,NaN
4932,38.937437,-76.971361,3902 25TH PLACE NE,20018,Woodridge,Cluster 24,Ward 5,3303,0.224947,"38.9374374,-76.97136058",https://mobilityscore.transitscreen.io/api/v1/...,[{'data': {'mobilityScore': {'latitude': 38.93...,NaN


In [22]:
print(df.shape)

(4933, 13)


In [24]:
df.dropna(subset = ['json'], inplace=True) # Delete them.
print(df.shape)

(4929, 13)


In [34]:
for i in df.index:
    df.loc[(df.index==i) , 'mobilityscore']=df['json'][i][0]['data']['mobilityScore']['score']

KeyError: 'data'

In [35]:
df['mobilityscore'].isnull().sum()

2212

In [37]:
df.dropna(subset = ['mobilityscore'], inplace=True) # Delete them.
print(df.shape)

(2717, 13)


In [38]:
df.drop(['call', 'json'], axis=1, inplace=True)
df = df[['coordinates', 'LATITUDE', 'LONGITUDE', 'FULLADDRESS', 'ZIPCODE', 'ASSESSMENT_NBHD', 'CLUSTER_', 'WARD', 'Units', 'Residential', 'mobilityscore']]
df.head()

,coordinates,LATITUDE,LONGITUDE,FULLADDRESS,ZIPCODE,ASSESSMENT_NBHD,CLUSTER_,WARD,Units,Residential,mobilityscore
0,"38.8688316,-77.01825188",38.868832,-77.018252,221 2ND AVENUE SW,20024,Ft. McNair,Cluster 9,Ward 6,59,0.20339,54.0
1,"38.8675634,-77.01823116",38.867563,-77.018231,241 2ND AVENUE SW,20024,Ft. McNair,Cluster 9,Ward 6,59,0.20339,49.0
2,"38.90749873,-76.94332398",38.907499,-76.943324,4300 ANACOSTIA AVENUE NE,20019,DC Stadium Area,Cluster 29,Ward 7,57,0.00000,53.0
3,"38.87882265,-76.97541985",38.878823,-76.975420,1900 M STREET SE,20003,Anacostia Park,Cluster 26,Ward 6,16,0.00000,49.0
4,"38.87269412,-76.99086222",38.872694,-76.990862,1105 O STREET SE,20003,Anacostia Park,Cluster 27,Ward 6,16,0.00000,71.0


In [39]:
df.to_csv('data/dc_mscores.csv',index=False)